# Text 3: Latent Dirichlet allocation
**Internet Analytics - Lab 4**

---

**Group:** *H*

**Names:**

* *Jérémy Baffou*
* *Antoine Basseto*
* *Andrea Pinto*

---

- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.mllib.clustering.LDA.html
- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.mllib.clustering.LDAModel.html

In [1]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors, SparseVector
from numpy.testing import assert_almost_equal, assert_equal
from utils import load_json
import numpy as np

In [2]:
courses_preprocessed = load_json('courses-lem-preprocessed.json')
courses_preprocessed_rdd = sc.parallelize(courses_preprocessed)

## Minor Preprocessing

In [3]:
def populate(data, n, m):
    tc = np.zeros((n, m))
    for x in data:
        infos = x[0]
        value = x[1]
        
        doc_id = infos[0]
        word_id = infos[1]
        
        tc[doc_id][word_id] = value
    
    return tc

In [4]:
def bow(rdd):
    return set(rdd.flatMap(lambda c: c["description"]).collect())

In [5]:
superbow = bow(courses_preprocessed_rdd)
words_ids_map = dict(zip(superbow, range(len(superbow))))

In [6]:
courses_ids = courses_preprocessed_rdd.map(lambda c: c['courseId']).collect()
doc_ids_map = dict(zip(courses_ids, range(len(courses_ids))))
doc_ids = list(doc_ids_map.values())

In [7]:
termcount = (
    courses_preprocessed_rdd
    .map(lambda c: [((doc_ids_map[c["courseId"]], words_ids_map[w]),1) for w in c["description"]])
    .flatMap(lambda c: c)
    .reduceByKey(lambda x, y : x + y)
)

In [8]:
tc = populate(termcount.collect(), len(doc_ids), len(superbow))

In [9]:
tc.shape

(854, 11641)

## Exercise 4.8: Topics extraction

In [34]:
doc_tc_tup = list(zip(doc_ids, [Vectors.dense(wordcount) for wordcount in tc]))
doc_tc = list(map(lambda x: list(x), doc_tc_tup))

In [38]:
rdd = sc.parallelize(doc_tc)

In [39]:
model = LDA.train(rdd, k=10, seed=1)

## Exercise 4.9: Dirichlet hyperparameters

## Exercise 4.10: EPFL's taught subjects

## Exercise 4.11: Wikipedia structure